In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

from sklearn.model_selection import train_test_split
import csv

In [2]:
dataset = np.load('./data_train.npy').T
labels = np.load('./labels_train.npy')
data_train, data_val, labels_train, labels_val = train_test_split(dataset, 
                                                                    labels,
                                                                    test_size=0.2,
                                                                    shuffle=True,
                                                                    random_state=0)
data_train.shape

(4746, 270000)

#### Reshaping

In [5]:
data_train_size = data_train.shape[0]
data_train_shaped = np.zeros(shape=(data_train_size,300,300,3))
for image in range(data_train_size):
    one_img = data_train[image].reshape((300, 300, 3))
    data_train_shaped[image] = one_img
data_train_shaped.shape

data_val_size = data_val.shape[0]
data_val_shaped = np.zeros(shape=(data_val_size,300,300,3))
for image in range(data_val_size):
    one_img = data_val[image].reshape((300, 300, 3))
    data_val_shaped[image] = one_img

data_train_shaped.shape, data_val_shaped.shape

((4746, 300, 300, 3), (1187, 300, 300, 3))

In [6]:
import gc
del dataset
del labels
del data_train
del data_val
gc.collect()

data_train_shaped = data_train_shaped.astype('uint8')
labels_train = labels_train.astype('uint8')
data_val_shaped = data_val_shaped.astype('uint8')
labels_val = labels_val.astype('uint8')

#### Reformatting

In [7]:
train_dataset = tf.data.Dataset.from_tensor_slices((data_train_shaped, labels_train))
val_dataset = tf.data.Dataset.from_tensor_slices((data_val_shaped, labels_val))
BATCH_SIZE = 42
SHUFFLE_BUFFER_SIZE = 100
train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
val_dataset = val_dataset.batch(BATCH_SIZE)

AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_dataset.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_dataset.cache().prefetch(buffer_size=AUTOTUNE)

2023-08-07 22:26:46.692075: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-07 22:26:47.233725: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 79111 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-80GB, pci bus id: 0000:90:00.0, compute capability: 8.0


In [8]:
train_steps_per_epoch=int(tf.data.experimental.cardinality(train_dataset))
val_steps_per_epoch=int(tf.data.experimental.cardinality(val_dataset))

del train_dataset
del val_dataset
gc.collect()

0

In [9]:
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal", input_shape=(300,300,3)),
    tf.keras.layers.RandomRotation(0.1), # Rotates Images +- 0.2rad (11.5deg)
    tf.keras.layers.RandomContrast(0.4), #Add contrast from .1 to 1 of original
    tf.keras.layers.RandomZoom(-0.6),  # Add RandomZoom layer for a change in perspective
])

In [10]:
# Create the base model from the pre-trained model MobileNet V2
IMG_SHAPE = (300, 300, 3)
#from tensorflow.keras.applications import EfficientNetB0

base_model = tf.keras.applications.EfficientNetB7(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

In [11]:
base_model.trainable = False

In [12]:
model = tf.keras.models.Sequential([
  data_augmentation,
  base_model,
  tf.keras.layers.Conv2D(32, 1, padding='same', activation='selu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, padding='same', activation='selu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(128, 5, padding='same', activation='selu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='selu'),
  tf.keras.layers.Dropout(0.5),
  tf.keras.layers.Dense(10, name="outputs")
])

In [13]:
model.compile(optimizer=keras.optimizers.Adam(learning_rate=.0001),#.0001 or less
          loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
          metrics=['accuracy'])

In [14]:
epochs=300
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs, 
  callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)]
)

Epoch 1/300


2023-08-07 22:27:43.530312: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8201
2023-08-07 22:27:46.127025: I tensorflow/stream_executor/cuda/cuda_blas.cc:1774] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


113/113 [==============================] - 38s 213ms/step - loss: 2.2479 - accuracy: 0.1909 - val_loss: 1.8284 - val_accuracy: 0.4987
Epoch 2/300
113/113 [==============================] - 21s 182ms/step - loss: 1.6257 - accuracy: 0.4579 - val_loss: 1.0798 - val_accuracy: 0.6908
Epoch 3/300
113/113 [==============================] - 21s 182ms/step - loss: 1.0789 - accuracy: 0.6477 - val_loss: 0.6803 - val_accuracy: 0.8088
Epoch 4/300
113/113 [==============================] - 21s 182ms/step - loss: 0.7764 - accuracy: 0.7545 - val_loss: 0.5300 - val_accuracy: 0.8399
Epoch 5/300
113/113 [==============================] - 21s 182ms/step - loss: 0.6194 - accuracy: 0.8051 - val_loss: 0.4528 - val_accuracy: 0.8551
Epoch 6/300
113/113 [==============================] - 21s 182ms/step - loss: 0.5591 - accuracy: 0.8312 - val_loss: 0.3902 - val_accuracy: 0.8753
Epoch 7/300
113/113 [==============================] - 21s 182ms/step - loss: 0.4929 - accuracy: 0.8458 - val_loss: 0.3639 - val_accurac

In [15]:
# second pass
model.compile(optimizer=keras.optimizers.Adam(learning_rate=.00001),#.0001 or less
          loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
          metrics=['accuracy'])

epochs=300
history = model.fit(
  train_ds,
  validation_data=val_ds,
  initial_epoch=history.epoch[-1],
  epochs=epochs, 
  callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)]
)

Epoch 55/300
113/113 [==============================] - 34s 207ms/step - loss: 0.1517 - accuracy: 0.9520 - val_loss: 0.2047 - val_accuracy: 0.9360
Epoch 56/300
113/113 [==============================] - 20s 181ms/step - loss: 0.1453 - accuracy: 0.9528 - val_loss: 0.2048 - val_accuracy: 0.9368
Epoch 57/300
113/113 [==============================] - 21s 182ms/step - loss: 0.1482 - accuracy: 0.9515 - val_loss: 0.2029 - val_accuracy: 0.9360
Epoch 58/300
113/113 [==============================] - 21s 182ms/step - loss: 0.1400 - accuracy: 0.9543 - val_loss: 0.2016 - val_accuracy: 0.9368
Epoch 59/300
113/113 [==============================] - 20s 181ms/step - loss: 0.1330 - accuracy: 0.9560 - val_loss: 0.2030 - val_accuracy: 0.9360
Epoch 60/300
113/113 [==============================] - 20s 181ms/step - loss: 0.1448 - accuracy: 0.9545 - val_loss: 0.2025 - val_accuracy: 0.9385
Epoch 61/300
113/113 [==============================] - 21s 182ms/step - loss: 0.1373 - accuracy: 0.9564 - val_loss: 0

In [27]:
loss, acc = model.evaluate(val_ds, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

29/29 - 4s - loss: 0.1971 - accuracy: 0.9410 - 4s/epoch - 139ms/step
Restored model, accuracy: 94.10%


In [29]:
loss, acc = model.evaluate(train_ds, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

113/113 - 16s - loss: 0.0488 - accuracy: 0.9853 - 16s/epoch - 141ms/step
Restored model, accuracy: 98.53%


In [30]:
base_model.trainable = True

# Let's take a look to see how many layers are in the base model
print("Number of layers in the base model: ", len(base_model.layers))

Number of layers in the base model:  813


In [33]:
# Fine-tune from this layer onwards
fine_tune_at = 700

learning_rate = 0.0005

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable = False

In [35]:
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0005),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
epochs=20
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs,
  callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)]
)

Epoch 1/20
113/113 [==============================] - 38s 237ms/step - loss: 0.1869 - accuracy: 0.9610 - val_loss: 0.4280 - val_accuracy: 0.9309
Epoch 2/20
113/113 [==============================] - 24s 212ms/step - loss: 0.1806 - accuracy: 0.9610 - val_loss: 0.2968 - val_accuracy: 0.9419
Epoch 3/20
113/113 [==============================] - 24s 212ms/step - loss: 0.1048 - accuracy: 0.9741 - val_loss: 0.1809 - val_accuracy: 0.9621
Epoch 4/20
113/113 [==============================] - 24s 210ms/step - loss: 0.0830 - accuracy: 0.9806 - val_loss: 0.1945 - val_accuracy: 0.9604
Epoch 5/20
113/113 [==============================] - 24s 212ms/step - loss: 0.1058 - accuracy: 0.9753 - val_loss: 0.1791 - val_accuracy: 0.9663
Epoch 6/20
113/113 [==============================] - 24s 211ms/step - loss: 0.0687 - accuracy: 0.9834 - val_loss: 0.2022 - val_accuracy: 0.9638
Epoch 7/20
113/113 [==============================] - 24s 212ms/step - loss: 0.0593 - accuracy: 0.9855 - val_loss: 0.1726 - val_ac

In [36]:
model.save('NocturnaEmblemClassifierV3.h5', save_format='h5')

/apps/tensorflow/2.7.0/lib/python3.9/site-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


In [26]:
mymodel.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (None, 300, 300, 3)       0         
                                                                 
 efficientnetb7 (Functional)  (None, 10, 10, 2560)     64097687  
                                                                 
 conv2d (Conv2D)             (None, 10, 10, 32)        81952     
                                                                 
 dropout (Dropout)           (None, 10, 10, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 5, 5, 32)         0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 5, 5, 32)          9248      
                                                      

In [38]:
loss, acc = model.evaluate(val_ds, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

29/29 - 4s - loss: 0.1400 - accuracy: 0.9747 - 4s/epoch - 139ms/step
Restored model, accuracy: 97.47%


In [39]:
loss, acc = model.evaluate(train_ds, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

113/113 - 16s - loss: 0.0116 - accuracy: 0.9977 - 16s/epoch - 141ms/step
Restored model, accuracy: 99.77%
